# Identifier un SPAM

Le but de cette exercice est d'identifier automatiquement un spam ou un email normal 

Nous passerons par les étapes suivantes : 

1. Nettoyage de la données.
2. Convertir le texte nettoyé en une representation numérique.
3. Appliquer un model **Multinomial Naive Bayes**, pour classifier chaque un email en spam ou en non spam.

## La librairie NLTK (Natural Language Toolkit)

Installer et/ou importer la librairie NLTK

In [1]:
#pip install nltk


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/jerome/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jerome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jerome/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jerome/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import pandas as pd

df = pd.read_csv("./data/emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
df.tail()

,text,spam
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0
5727,Subject: news : aurora 5 . 2 update aurora ve...,0


## (1) Cleaning du text du dataset

L'ensemble de données est composé d'e-mails classés comme ham [0] ou spam[1]. Nous devons nettoyer les données avant de former un modèle de prédiction.

### (1.1) Supprimer la Ponctuation

Créer une fonction pour supprimer la ponctuation. Appliquez-le à la colonne de texte et ajoutez la sortie à une nouvelle colonne dans le dataframe appelé clean_text


**La sortie devrait ressembler à ça :**

<img src='./images/1.png'>

In [3]:
import string
ls_punct = string.punctuation

def remove_ponctuation(email, ls_punct):
    for punct in ls_punct:
        email = email.replace(punct,'')
    return email

In [14]:
df['text'] = df['text'].apply(lambda x: remove_ponctuation(x, ls_punct))

### (1.2) Lower Case

Créer une fonction pour mettre le texte en minuscules. Nous l'appliquons à `clean_text`

**La sortie devrait ressembler à ça :**

<img src="./images/2.png">

In [15]:
df['clean_text'] = df['text'].apply(lambda x: x.lower())

In [16]:
df['clean_text']

0       subject naturally irresistible your corporate ...
1       subject the stock trading gunslinger  fanny is...
2       subject unbelievable new homes made easy  im w...
3       subject 4 color printing special  request addi...
4       subject do not have money  get software cds fr...
                              ...                        
5723    subject re  research and development charges t...
5724    subject re  receipts from visit  jim   thanks ...
5725    subject re  enron case study update  wow  all ...
5726    subject re  interest  david   please  call shi...
5727    subject news  aurora 5  2 update  aurora versi...
Name: clean_text, Length: 5728, dtype: object

### (1.3) Supprimer les nombres

Créer une fonction pour supprimer les nombres du texte. Nous l'appliquons à clean_text


**La sortie devrait ressembler à ça :**

<img src="./images/3.png">

In [17]:
def remove_numb(email):
    for numb in ['0','1','2','3','4','5','6','7','8','9']:
        email = email.replace(numb,'')
    return email        

In [18]:
df['clean_text'] = df['clean_text'].apply(lambda x: remove_numb(x))

### (1.4) Supprimer les StopWords

Créer une fonction pour supprimer les mots vide de sens. Nous l'appliquons à clean_text.


In [19]:
# Utiliser ces outils
from nltk.corpus import stopwords 
from nltk import word_tokenize


**La sortie devrait ressembler à ça :**

<img src="./images/4.png">

In [20]:
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/jerome/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
ls_sw = [i for i in stopwords.words('english')]
space = ' '
def remove_stopwords(email):
    output = ' '.join([i for i in word_tokenize(email) if i not in ls_sw])
    return output

In [22]:
df['clean_text'] = df['clean_text'].apply(lambda x: remove_stopwords(x))

In [23]:
df['clean_text'].loc[0]

'subject naturally irresistible corporate identity lt really hard recollect company market full suqgestions information isoverwhelminq good catchy logo stylish statlonery outstanding website make task much easier promise havinq ordered iogo company automaticaily become world ieader isguite ciear without good products effective business organization practicable aim hotat nowadays market promise marketing efforts become much effective list clear benefits creativeness hand made original logos specially done reflect distinctive company image convenience logo stationery provided formats easy use content management system letsyou change website content even structure promptness see logo drafts within three business days affordability marketing break make gaps budget satisfaction guaranteed provide unlimited amount changes extra fees surethat love result collaboration look portfolio interested'

### (1.5) Lemmatize

Créons une fonction pour lemmatiser le texte. Assurez-vous que la sortie est une chaîne unique, pas une liste de mots. Nous l'appliquons à clean_text


In [24]:
#Utiliser cet outil
from nltk.stem import WordNetLemmatizer


In [25]:
def lemmatize_email(email):
    lemmatizer = WordNetLemmatizer()
    output = []
    for word in email.split(' '):
        output.append(lemmatizer.lemmatize(word))
    return ' '.join(output)

In [26]:
df['clean_text'] = df['clean_text'].apply(lambda x : lemmatize_email(x))

**La sortie devrait ressembler à cela :**

<img src="./images/5.png">

## (2) CountVectorize

### (2.1) Numérisation des données textuelles en chiffres

Vectoriser clean_text avec un CountVectorize et laisser les paramatres par défault.

In [27]:
#Utiliser cet outil
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df.clean_text)
len(vectorizer.get_feature_names_out())
X.shape

(5728, 30962)

In [29]:
df_X = pd.DataFrame.sparse.from_spmatrix(X, index=None, columns=None)

In [30]:
df_X

,0,1,2,3,4,5,6,7,8,9,...,30952,30953,30954,30955,30956,30957,30958,30959,30960,30961
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5724,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df_y = df['spam']

**La sortie devrait ressembler à ça :**

<img src="./images/6.png">

### (2.2) Multinomial Naive Bayes Modelling

Cross valider le model et scorer le model avec l'accuracy metrique.

**Vous devriez obtenir un score proche de 1.**

In [32]:
#Utiliser ces outils
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate


In [47]:
clf = MultinomialNB(force_alpha=True)

In [48]:
cv_model = cross_validate(clf, df_X, df_y, cv = 5)

In [35]:
cv_model['test_score']

array([0.98691099, 0.9895288 , 0.991274  , 0.98777293, 0.99213974])

<h3>The cross validation of the model show accuracy around 99%</h3>

<h2>Let's see the model on a personal spam</h2>

In [78]:
df.head(5)

,text,spam,clean_text
0,Subject naturally irresistible your corporate ...,1,subject naturally irresistible corporate ident...
1,Subject the stock trading gunslinger fanny is...,1,subject stock trading gunslinger fanny merrill...
2,Subject unbelievable new homes made easy im w...,1,subject unbelievable new home made easy im wan...
3,Subject 4 color printing special request addi...,1,subject color printing special request additio...
4,Subject do not have money get software cds fr...,1,subject money get software cd software compati...


In [50]:
model = clf.fit(df_X, df_y)

In [60]:
my_spam = '''Subject Jerome Here is the deal Jerome.

I need to bring on 3 people this month.

You seem like a great fit

The invite is attached, the password is 2343

I look forward to hearing from you Jerome.'''

In [67]:
my_spam = df['text'].loc[0]

In [74]:
my_spam

'subject naturally irresistible corporate identity lt really hard recollect company market full suqgestions information isoverwhelminq good catchy logo stylish statlonery outstanding website make task much easier promise havinq ordered iogo company automaticaily become world ieader isguite ciear without good product effective business organization practicable aim hotat nowadays market promise marketing effort become much effective list clear benefit creativeness hand made original logo specially done reflect distinctive company image convenience logo stationery provided format easy use content management system letsyou change website content even structure promptness see logo draft within three business day affordability marketing break make gap budget satisfaction guaranteed provide unlimited amount change extra fee surethat love result collaboration look portfolio interested'

In [75]:
#running the email through the preprocessing pipeline
my_spam = remove_ponctuation(my_spam, ls_punct)
my_spam = my_spam.lower()
my_spam = remove_numb(my_spam)
my_spam = remove_stopwords(my_spam)
my_spam = lemmatize_email(my_spam)
res = vectorizer.transform([my_spam])

In [76]:
pred = model.predict(res.toarray())

In [77]:
pred

array([1])